## Importing the Libraries

In [1]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

## Getting the data

In [2]:
seq_hotel = "datasets/ewap_dataset/seq_hotel"
seq_eth   = "datasets/ewap_dataset/seq_eth"

hotel_obsmat = seq_hotel + "/obsmat.txt"

In [3]:
f_obsmat = open(hotel_obsmat,"r")

In [4]:
f_obsmat.seek(0)
len(f_obsmat.readlines())

6544

The actual annotation is stored in the obsmat.txt file. Each line has this format

*[frame_number pedestrian_ID pos_x pos_z pos_y v_x v_z v_y ]*

In [5]:
df_obsmat = pd.read_csv(hotel_obsmat, delimiter=r"\s+", header = None)
df_obsmat.columns = ["frame_number", "pedestrian_ID", "pos_x", "pos_z", "pos_y", "v_x", "v_z", "v_y"]
df_obsmat = df_obsmat.drop(["pos_z", "v_z"], axis = 1)

In [6]:
pedestrian = df_obsmat[df_obsmat.pedestrian_ID == 25]
list(pedestrian.pos_x)

[-0.65218834,
 -0.43194664,
 -0.27115282,
 -0.072714103,
 0.065694252,
 0.16238537,
 0.27899983,
 0.26068875,
 0.30032937,
 0.26308032,
 0.26370402,
 0.30107617,
 0.2592225,
 0.3511434,
 0.36728377,
 0.42126177,
 0.51411648,
 0.54798662,
 0.67720055,
 0.71214431,
 0.80459493,
 0.89691052,
 0.87550639,
 0.94675676,
 0.92424993,
 0.9767384,
 0.99398095,
 0.97261657,
 1.0835117,
 1.1392725,
 1.2334458]

In [7]:
len(df_obsmat.frame_number.unique())

1168

In [8]:
df_obsmat.head

<bound method NDFrame.head of       frame_number  pedestrian_ID     pos_x     pos_y       v_x       v_y
0              1.0            1.0  1.398378 -5.743303 -0.327083 -1.680286
1              1.0            2.0  0.517796 -7.003832  0.097997 -1.629874
2              1.0            3.0  2.259814 -4.546597 -0.901111  1.187025
3              1.0            4.0  2.740179 -2.423851  0.053061  1.815625
4              1.0            5.0 -1.585502  0.927584  0.000000  0.000000
...            ...            ...       ...       ...       ...       ...
6539       18051.0          420.0  3.617401 -6.192644  0.014910  1.543804
6540       18061.0          416.0  1.132991 -8.158643 -0.034447 -1.460363
6541       18061.0          417.0  2.695246 -8.296147  0.194425 -1.452633
6542       18061.0          419.0  3.353155 -1.422939  0.002049  1.712019
6543       18061.0          420.0  3.615008 -5.564871 -0.005983  1.569430

[6544 rows x 6 columns]>

In [9]:
missing = []
prev = -1
for x in df_obsmat.frame_number.unique():
    x -= 1
    x = int(x/10)
    if x != prev + 1:
        for i in range(prev+1, x):
            missing.append(i)
            
    prev = x
            
missing

[36,
 37,
 38,
 39,
 40,
 82,
 83,
 84,
 85,
 86,
 207,
 208,
 209,
 225,
 226,
 227,
 228,
 229,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 299,
 300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 502,
 503,
 504,
 505,
 506,
 507,
 508,
 509,
 510,
 511,
 512,
 513,
 514,
 515,
 516,
 517,
 518,
 519,
 520,
 521,
 522,
 523,
 538,
 539,
 540,
 541,
 542,
 543,
 544,
 545,
 546,
 5

In [10]:
for i in range(len(df_obsmat)):
    df_obsmat.iloc[i,0] -= 1
    df_obsmat.iloc[i,0] = int(df_obsmat.iloc[i,0]/10)
#     print(df_obsmat.iloc[i,0])

In [11]:
df_obsmat = df_obsmat.astype({'frame_number': int, 'pedestrian_ID' : int})

In [12]:
df_obsmat

,frame_number,pedestrian_ID,pos_x,pos_y,v_x,v_y
0,0,1,1.398378,-5.743303,-0.327083,-1.680286
1,0,2,0.517796,-7.003832,0.097997,-1.629874
2,0,3,2.259814,-4.546597,-0.901111,1.187025
3,0,4,2.740179,-2.423851,0.053061,1.815625
4,0,5,-1.585502,0.927584,0.000000,0.000000
...,...,...,...,...,...,...
6539,1805,420,3.617401,-6.192644,0.014910,1.543804
6540,1806,416,1.132991,-8.158643,-0.034447,-1.460363
6541,1806,417,2.695246,-8.296147,0.194425,-1.452633
6542,1806,419,3.353155,-1.422939,0.002049,1.712019


In [13]:
df_obsmat.to_csv("datasets/ewap_dataset/obsmat_hotel.csv")